In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import numpy as np
import pandas as pd
from pathlib import Path
from collections import Counter

In [3]:
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import confusion_matrix
from imblearn.metrics import classification_report_imbalanced

# Read the CSV and Perform Basic Data Cleaning

In [4]:
# https://help.lendingclub.com/hc/en-us/articles/215488038-What-do-the-different-Note-statuses-mean-

columns = [
    "loan_amnt", "int_rate", "installment", "home_ownership",
    "annual_inc", "verification_status", "issue_d", "loan_status",
    "pymnt_plan", "dti", "delinq_2yrs", "inq_last_6mths",
    "open_acc", "pub_rec", "revol_bal", "total_acc",
    "initial_list_status", "out_prncp", "out_prncp_inv", "total_pymnt",
    "total_pymnt_inv", "total_rec_prncp", "total_rec_int", "total_rec_late_fee",
    "recoveries", "collection_recovery_fee", "last_pymnt_amnt", "next_pymnt_d",
    "collections_12_mths_ex_med", "policy_code", "application_type", "acc_now_delinq",
    "tot_coll_amt", "tot_cur_bal", "open_acc_6m", "open_act_il",
    "open_il_12m", "open_il_24m", "mths_since_rcnt_il", "total_bal_il",
    "il_util", "open_rv_12m", "open_rv_24m", "max_bal_bc",
    "all_util", "total_rev_hi_lim", "inq_fi", "total_cu_tl",
    "inq_last_12m", "acc_open_past_24mths", "avg_cur_bal", "bc_open_to_buy",
    "bc_util", "chargeoff_within_12_mths", "delinq_amnt", "mo_sin_old_il_acct",
    "mo_sin_old_rev_tl_op", "mo_sin_rcnt_rev_tl_op", "mo_sin_rcnt_tl", "mort_acc",
    "mths_since_recent_bc", "mths_since_recent_inq", "num_accts_ever_120_pd", "num_actv_bc_tl",
    "num_actv_rev_tl", "num_bc_sats", "num_bc_tl", "num_il_tl",
    "num_op_rev_tl", "num_rev_accts", "num_rev_tl_bal_gt_0",
    "num_sats", "num_tl_120dpd_2m", "num_tl_30dpd", "num_tl_90g_dpd_24m",
    "num_tl_op_past_12m", "pct_tl_nvr_dlq", "percent_bc_gt_75", "pub_rec_bankruptcies",
    "tax_liens", "tot_hi_cred_lim", "total_bal_ex_mort", "total_bc_limit",
    "total_il_high_credit_limit", "hardship_flag", "debt_settlement_flag"
]

target = ["loan_status"]

In [5]:
# Load the data
file_path = Path('LoanStats_2019Q1.csv')
df = pd.read_csv(file_path, skiprows=1)[:-2]
df = df.loc[:, columns].copy()

# Drop the null columns where all values are null
df = df.dropna(axis='columns', how='all')

# Drop the null rows
df = df.dropna()

# Remove the `Issued` loan status
issued_mask = df['loan_status'] != 'Issued'
df = df.loc[issued_mask]

# convert interest rate to numerical
df['int_rate'] = df['int_rate'].str.replace('%', '')
df['int_rate'] = df['int_rate'].astype('float') / 100


# Convert the target column values to low_risk and high_risk based on their values
x = {'Current': 'low_risk'}   
df = df.replace(x)

x = dict.fromkeys(['Late (31-120 days)', 'Late (16-30 days)', 'Default', 'In Grace Period'], 'high_risk')    
df = df.replace(x)

df.reset_index(inplace=True, drop=True)

df.head()

,loan_amnt,int_rate,installment,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,dti,...,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,hardship_flag,debt_settlement_flag
0,10500.0,0.1719,375.35,RENT,66000.0,Source Verified,Mar-2019,low_risk,n,27.24,...,85.7,100.0,0.0,0.0,65687.0,38199.0,2000.0,61987.0,N,N
1,25000.0,0.2000,929.09,MORTGAGE,105000.0,Verified,Mar-2019,low_risk,n,20.23,...,91.2,50.0,1.0,0.0,271427.0,60641.0,41200.0,49197.0,N,N
2,20000.0,0.2000,529.88,MORTGAGE,56000.0,Verified,Mar-2019,low_risk,n,24.26,...,66.7,50.0,0.0,0.0,60644.0,45684.0,7500.0,43144.0,N,N
3,10000.0,0.1640,353.55,RENT,92000.0,Verified,Mar-2019,low_risk,n,31.44,...,100.0,50.0,1.0,0.0,99506.0,68784.0,19700.0,76506.0,N,N
4,22000.0,0.1474,520.39,MORTGAGE,52000.0,Not Verified,Mar-2019,low_risk,n,18.76,...,100.0,0.0,0.0,0.0,219750.0,25919.0,27600.0,20000.0,N,N


# Split the Data into Training and Testing

In [6]:
df1 = pd.get_dummies(df,dtype=float)
df1.columns

Index(['loan_amnt', 'int_rate', 'installment', 'annual_inc', 'dti',
       'delinq_2yrs', 'inq_last_6mths', 'open_acc', 'pub_rec', 'revol_bal',
       'total_acc', 'out_prncp', 'out_prncp_inv', 'total_pymnt',
       'total_pymnt_inv', 'total_rec_prncp', 'total_rec_int',
       'total_rec_late_fee', 'recoveries', 'collection_recovery_fee',
       'last_pymnt_amnt', 'collections_12_mths_ex_med', 'policy_code',
       'acc_now_delinq', 'tot_coll_amt', 'tot_cur_bal', 'open_acc_6m',
       'open_act_il', 'open_il_12m', 'open_il_24m', 'mths_since_rcnt_il',
       'total_bal_il', 'il_util', 'open_rv_12m', 'open_rv_24m', 'max_bal_bc',
       'all_util', 'total_rev_hi_lim', 'inq_fi', 'total_cu_tl', 'inq_last_12m',
       'acc_open_past_24mths', 'avg_cur_bal', 'bc_open_to_buy', 'bc_util',
       'chargeoff_within_12_mths', 'delinq_amnt', 'mo_sin_old_il_acct',
       'mo_sin_old_rev_tl_op', 'mo_sin_rcnt_rev_tl_op', 'mo_sin_rcnt_tl',
       'mort_acc', 'mths_since_recent_bc', 'mths_since_recent_in

In [7]:
# Create our features
# Create our target

y = df1['loan_status_high_risk']
X = df1.drop(columns=['loan_status_high_risk', 'loan_status_low_risk'])
target_names = ['low risk', 'high risk']

In [8]:
# Check the balance of our target values
y.value_counts()

0.0    68470
1.0      347
Name: loan_status_high_risk, dtype: int64

In [9]:
from sklearn.model_selection import train_test_split
# YOUR CODE HERE
X_train, X_test, y_train, y_test = train_test_split (X, y, random_state=1)
Counter(y_train)

Counter({0.0: 51366, 1.0: 246})

# Ensemble Learners

In this section, you will compare two ensemble algorithms to determine which algorithm results in the best performance. You will train a Balanced Random Forest Classifier and an Easy Ensemble AdaBoost classifier . For each algorithm, be sure to complete the folliowing steps:

1. Train the model using the training data. 
2. Calculate the balanced accuracy score from sklearn.metrics.
3. Print the confusion matrix from sklearn.metrics.
4. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.
5. For the Balanced Random Forest Classifier onely, print the feature importance sorted in descending order (most important feature to least important) along with the feature score

Note: Use a random state of 1 for each algorithm to ensure consistency between tests

### Balanced Random Forest Classifier

In [14]:
# Resample the training data with the BalancedRandomForestClassifier
# YOUR CODE HERE
from imblearn.ensemble import BalancedRandomForestClassifier
clf = BalancedRandomForestClassifier(n_estimators=100, random_state=1).fit(X_train,y_train)
y_pred = clf.predict(X_test)

In [15]:
print(f'------ Balanced Random Forest Classifier ------')
print(f'Balanced accuracy score: {balanced_accuracy_score(y_test, y_pred)}')
print(f'-----------------------------------------------')
print(classification_report_imbalanced(y_test, y_pred, target_names=target_names))
print(f'---confusion matrix------')
pd.DataFrame(confusion_matrix(y_test, y_pred), columns=['Positive', 'Negative'], index=['low risk','high risk'])

------ Balanced Random Forest Classifier ------
Balanced accuracy score: 0.7665611772823682
-----------------------------------------------
                   pre       rec       spe        f1       geo       iba       sup

   low risk       1.00      0.89      0.64      0.94      0.76      0.59     17104
  high risk       0.03      0.64      0.89      0.06      0.76      0.56       101

avg / total       0.99      0.89      0.65      0.94      0.76      0.59     17205

---confusion matrix------


,Positive,Negative
low risk,15215,1889
high risk,36,65


In [16]:
# print feature importance sorted in descending order
features = sorted(zip(clf.feature_importances_, X.columns), reverse=True)
for f in features:
    print('{} : {}'.format(f[1], f[0]))

total_rec_prncp : 0.08200603138527934
total_pymnt_inv : 0.06396934747748335
total_rec_int : 0.05744347415935885
total_pymnt : 0.05059089525220024
last_pymnt_amnt : 0.04957836333074211
int_rate : 0.03313864789639978
issue_d_Jan-2019 : 0.02047701317088599
installment : 0.0193697462184253
out_prncp_inv : 0.017439180890896647
tot_hi_cred_lim : 0.016900289617929845
dti : 0.016779159090776104
mths_since_rcnt_il : 0.015918154332127777
annual_inc : 0.015730979750990595
total_bc_limit : 0.015019425852185124
mo_sin_old_il_acct : 0.014632588232213887
avg_cur_bal : 0.014531434631017947
mths_since_recent_inq : 0.014476697543636563
bc_util : 0.014297199923957386
il_util : 0.014065599151397613
max_bal_bc : 0.01372024744521318
tot_cur_bal : 0.013672087318338818
total_rev_hi_lim : 0.013640477255942788
loan_amnt : 0.01358415277298156
mo_sin_old_rev_tl_op : 0.013197281637035508
revol_bal : 0.013188672806592725
mths_since_recent_bc : 0.012772403628715392
all_util : 0.01262996265448217
out_prncp : 0.012607

### Easy Ensemble AdaBoost Classifier

In [ ]:
#pip install -U scikit-learn
!pip install scikit-learn==1.0 -U 

In [ ]:
import sklearn
print('The scikit-learn version is {}.'.format(sklearn.__version__))

In [ ]:
!python -m pip show scikit-learn

In [10]:
# Train the EasyEnsembleClassifier
from imblearn.ensemble import EasyEnsembleClassifier
eec = EasyEnsembleClassifier(n_estimators=100, random_state=1)
eec.fit(X_train, y_train) 

EasyEnsembleClassifier(n_estimators=100, random_state=1)

In [12]:
y_pred = eec.predict(X_test)
Counter(y_pred)

Counter({0.0: 16129, 1.0: 1076})

In [13]:
print(f'------Easy Ensemble Classifier ------')
print(f'Balanced accuracy score: {balanced_accuracy_score(y_test, y_pred)}')
print(f'-----------------------------------------------')
print(classification_report_imbalanced(y_test, y_pred, target_names=target_names))
print(f'---confusion matrix------')
pd.DataFrame(confusion_matrix(y_test, y_pred), columns=['Positive', 'Negative'], index=['low risk','high risk'])

------Easy Ensemble Classifier ------
Balanced accuracy score: 0.9316600714093861
-----------------------------------------------
                   pre       rec       spe        f1       geo       iba       sup

   low risk       1.00      0.94      0.92      0.97      0.93      0.87     17104
  high risk       0.09      0.92      0.94      0.16      0.93      0.87       101

avg / total       0.99      0.94      0.92      0.97      0.93      0.87     17205

---confusion matrix------


,Positive,Negative
low risk,16121,983
high risk,8,93
